## **Import Libraries and Prediction Results**

In [ ]:
directory='/content' ### Set Directory

In [ ]:
%%bash
# Script to make sure data files can be used

directory='/content' ### Must be the same as the variable directory

if [ ! -d $directory ]; then
mkdir -p $directory
fi

cd $directory
wget https://storage.googleapis.com/yantk-siads-699/699_final.tar.gz
tar -xvzf ${directory}/699_final.tar.gz .
mkdir ${directory}/699
mkdir ${directory}/699/699
mv ${directory}/Data\ 20230612 ${directory}/699/699
mv ${directory}/model ${directory}/699/699

In [ ]:
from operator import itemgetter
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import plotly.express as px
from nltk.translate.bleu_score import sentence_bleu
import torch.nn.functional as F
from torch import cuda
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import json

In [ ]:
test_df = pd.read_csv(directory + '/699/699/Data 20230612/Youtube/test_set_youtube.csv', lineterminator='\n', converters={'top_tags': eval})
test_df

,Unnamed: 0,video_id,title,description,en_tag_list,top_tags
0,62932,ZqK2As0qimM,love.,Provided to YouTube by Universal Music Grouplo...,"['kid cudi', 'the boy who flew to the moon', '...",[kid cudi]
1,51926,ChiXMWEFBe8,Pope & Root Masterclass! | Highlights | Englan...,Go to ecb.co.uk to join We Are England Cricket...,"['england', 'england vs. new zealand', 'new ze...","[england, england vs. new zealand, new zealand..."
2,13291,cT7MNBaBvAk,telling you the truth.,this is the truth. ☂️ Shop Metal Umbrella: htt...,"['fortnite', 'fortnite battle royale', 'compet...","[fortnite, fortnite battle royale, competitive..."
3,22471,HSO3zOj-LDk,Basketball Mood Lifters! Part 7! 😆 #shorts,Basketball Mood Lifters! Part 7! 😆#shorts #you...,"['shorts', 'youtube shorts', 'tiktok', 'viral'...","[shorts, youtube shorts, tiktok, viral, instag..."
4,21000,GvsshVK5zRw,BRYNLEY IS PREGNANT!,BRYNLEY IS PREGNANT!Subscribe: https://bit.ly/...,"['brynley is pregnant', 'brynley pregnant', 'b...","[first pregnancy, fertility, baby announcement..."
...,...,...,...,...,...,...
5494,22509,k2C5kDAYSk0,Macron à Poutine : Tu te racontes des histoire...,"► Pour suivre LCI en direct, c'est ici : http:...","['emmanuel macron', 'emmanuel macron vladimir ...","[emmanuel macron, guerre ukraine, information,..."
5495,5310,Scjj9VZyhXU,"Soolking feat. Lynda, Heuss, L'Algérino, Frang...",Ecouter Jennifer Remix : https://soolking.lnk....,"['soolking', 'soolking officiel', 'soolking 20...","[soolking, soolking officiel, soolking 2020, s..."
5496,57265,F0cAUTIiLAA,Trump delivers acceptance speech at the RNC | ...,President Trump delivered his acceptance speec...,"['fox business', 'fox business live', 'fox bus...","[fox business, fox business live, fox business..."
5497,13202,kqXZDJf8t_k,Ryan Whitney & Paul Bissonnette Went Head To H...,Ever see a pair of pigeons grilling chicken?” ...,"['spittin chiclets', 'chiclets', 'hockey', 'nh...","[spittin chiclets, chiclets, hockey, nhl, ryan..."


In [ ]:
test_df["description"] = test_df["description"].astype(str)
test_df["title"] = test_df["title"].astype(str)

test_df["combined"] = test_df["title"] + '' + test_df["description"]

In [ ]:
test_df = test_df[["combined",'top_tags']]
tmp = test_df['top_tags'][1]
tmp

['england',
 'england vs. new zealand',
 'new zealand',
 'jonny bairstow',
 'england cricket',
 'daryl mitchell',
 'eng vs nz test highlights',
 'england new zealand cricket',
 'england cricket highlights',
 'england new zealand 2022',
 'england v new zealand',
 'eng vs nz test',
 'eng vs nz 2022',
 'england new zealand highlights',
 'eng vs nz highlights',
 'eng vs nz test highlights 2022',
 'england cricket match',
 'england cricket team',
 'england cricket vs new zealand',
 'cricket',
 'ben stokes',
 'ollie pope',
 'joe root']

In [ ]:
x = []
for tmp in test_df['top_tags']:
  for string in tmp:
    x.append(len(string.split()))
chart = pd.Series(x).value_counts()
chart

1     36597
2     26734
3     10517
4      3474
5      1119
6       346
7       134
8        47
9        16
10       13
12        7
11        6
14        3
dtype: int64

# **BLEU Score Analysis**

In [ ]:
px.bar(chart).update_layout(xaxis_title="Tag Length", yaxis_title="Frequency", width=1000)

In [ ]:
chart / chart.sum()

1     0.463177
2     0.338349
3     0.133105
4     0.043967
5     0.014162
6     0.004379
7     0.001696
8     0.000595
9     0.000202
10    0.000165
12    0.000089
11    0.000076
14    0.000038
dtype: float64

In [ ]:
bleu_weight = list(chart / chart.sum())
bleu_weight[3] = 1 - bleu_weight[0] - bleu_weight[1] - bleu_weight[2]
bleu_weight = bleu_weight[:4]
print('Sum of BLEU weight: ' + str(pd.Series(bleu_weight).sum()))
print('BLEU Weighting: ')
bleu_weight

Sum of BLEU weight: 1.0
BLEU Weighting: 


[0.46317694556591954,
 0.33834938554415095,
 0.13310467897687722,
 0.06536898991305223]

## **BLEU Weighting**

1-gram: 0.46317694556591954

2-gram: 0.33834938554415095

3-gram: 0.13310467897687722

4-gram: 0.06536898991305223

In [ ]:
dnn_512 = pd.read_csv(directory + '/699/699/Data 20230612/Youtube/dnn_results_512.csv', lineterminator='\n', converters={'dnn': eval})
dnn_768 = pd.read_csv(directory + '/699/699/Data 20230612/Youtube/dnn_results_768.csv', lineterminator='\n', converters={'dnn': eval})
dnn_1024 = pd.read_csv(directory + '/699/699/Data 20230612/Youtube/dnn_results_1024.csv', lineterminator='\n', converters={'dnn': eval})
vicuna = pd.read_csv(directory + '/699/699/Data 20230612/Youtube/vicuna_results.csv', lineterminator='\n')
gpt2 = pd.read_csv(directory + '/699/699/Data 20230612/Youtube/gpt2_results.csv', lineterminator='\n')

In [ ]:
result_df = test_df.copy()
result_df['dnn_512'] = dnn_512['dnn']
result_df['dnn_768'] = dnn_768['dnn']
result_df['dnn_1024'] = dnn_1024['dnn']
result_df['gpt2'] = gpt2['gpt2']
result_df['vicuna13b'] = vicuna['vicuna']
result_df

,combined,top_tags,dnn_512,dnn_768,dnn_1024,gpt2,vicuna13b
0,love.Provided to YouTube by Universal Music Gr...,[kid cudi],"[pop, music, records, savage, columbia, music ...","[kid cudi, boy better know, savage, spillage v...","[columbia, young thug, records, potter payper,...",['kid cudi'],"['love', 'Kid Cudi', 'The Boy Who Flew To The ..."
1,Pope & Root Masterclass! | Highlights | Englan...,"[england, england vs. new zealand, new zealand...","[england cricket, jonny bairstow, england cric...","[new zealand, england, cricket highlights, joe...","[england cricket board, cricket, cricket video...","['england', 'england vs. new zealand', 'new ze...","['cricket', 'test match', 'england cricket', '..."
2,telling you the truth.this is the truth. ☂️ Sh...,"[fortnite, fortnite battle royale, competitive...","[sypherpk, fortnite highlights, fortnite tips,...","[fortnite tips, sypherpk, fortnite highlights,...","[sypherpk, sypher, fortnite tips, how to play,...","['sypherpk', 'sypherpk', 'fortnite', 'sypherpk...","['sypherpk', 'fortnite', 'battle royale', 'gam..."
3,Basketball Mood Lifters! Part 7! 😆 #shortsBask...,"[shorts, youtube shorts, tiktok, viral, instag...","[ballislife, 2hype, jdub, basketball, kenny ch...","[nba, dunk, ballislife, basketball, tiktoker, ...","[kenny chao, ballislife, 2hype, 2k, jdub, gio ...","['dub', 'dub team', 'nba 2k', 'basketball', 'n...","['basketball', 'dunk', 'sports', 'viral', 'ins..."
4,BRYNLEY IS PREGNANT!BRYNLEY IS PREGNANT!Subscr...,"[first pregnancy, fertility, baby announcement...","[shorts, the ace family, trending, family, com...","[christian youtuber, natural labor, pranks, ru...","[london, vlogs, vlog, christian youtuber, mr b...","['family', 'shorts', 'shorts', 'shorts', 'lind...","['The Arnold Sisters', 'pregnancy announcement..."
...,...,...,...,...,...,...,...
5494,Macron à Poutine : Tu te racontes des histoire...,"[emmanuel macron, guerre ukraine, information,...","[sketch, humour, fun, clip, video, music, scen...","[information, russie, ukraine russie, tf1, fra...","[cinema, video drole, tournoi, inox, horreur, ...","['information', 'lci', 'russie', 'cartoon', 'a...","['politics', 'interview', 'french president', ..."
5495,"Soolking feat. Lynda, Heuss, L'Algérino, Frang...","[soolking, soolking officiel, soolking 2020, s...","[sofiane, dadju, zemer, melegim, soolking paro...","[soolking officiel, zemer, soolking parole, so...","[dadju, soolking album, soolking parole, soolk...","['soolking', 'jennifer', '","['Soolking', 'Lynda', 'Heuss', 'L\'Algérino', ..."
5496,Trump delivers acceptance speech at the RNC | ...,"[fox business, fox business live, fox business...","[news, world news, politics, donald trump, joe...","[politics, donald trump, crypto, joe biden, st...","[stock market, cable, breaking news, cable new...","['fox business', 'finance stock', 'stock marke...","['RNC', 'Republican National Convention', 'Tru..."
5497,Ryan Whitney & Paul Bissonnette Went Head To H...,"[spittin chiclets, chiclets, hockey, nhl, ryan...","[sports, podcast, hockey, barstool sports, nhl...","[comedy, barstool golf, biz, sports, paul biss...","[comedy, sports, barstool sports, podcast, bar...","['spittin chiclets', 'barstool sports', 'mr be...","['barstool sports', 'grilling competition', 'r..."


In [ ]:
broken = 0
def clean(string):
  """
  LLM has a limitation on output length, so add ending python list characters if needed
  Replace generated tags that contains aostrophe
  """
  global broken
  #lst = []
  string = string.lower()
  string = string.replace("['",'["')
  string = string.replace("']",'"]')
  string = string.replace("""', '""", '''", "''')
  string = string.replace('\n', '')
  string = string.replace('\r', '')
  if string[-2:] != '"]':
    if string[-1] != ',':
      string = string + '"]'
    if string[-1] == ',':
      string = string + ']'
  try:
    lst = eval(string)
  except:
    broken += 1
    lst = [].append("no_tag")
  return lst

result_df['gpt2_cleaned'] = result_df['gpt2'].apply(clean)
result_df['vicuna13b_cleaned'] = result_df['vicuna13b'].apply(clean)

<string>:1: SyntaxWarning:

list indices must be integers or slices, not tuple; perhaps you missed a comma?

<string>:1: SyntaxWarning:

list indices must be integers or slices, not tuple; perhaps you missed a comma?

<string>:1: SyntaxWarning:

list indices must be integers or slices, not tuple; perhaps you missed a comma?

<string>:1: SyntaxWarning:

list indices must be integers or slices, not tuple; perhaps you missed a comma?

<string>:1: SyntaxWarning:

list indices must be integers or slices, not tuple; perhaps you missed a comma?

<string>:1: SyntaxWarning:

list indices must be integers or slices, not tuple; perhaps you missed a comma?

<string>:1: SyntaxWarning:

list indices must be integers or slices, not tuple; perhaps you missed a comma?

<string>:1: SyntaxWarning:

list indices must be integers or slices, not tuple; perhaps you missed a comma?

<string>:1: SyntaxWarning:

list indices must be integers or slices, not tuple; perhaps you missed a comma?

<string>:1: SyntaxW

In [ ]:
reference = [' '.join(result_df['top_tags'][5498][:10]).split()]
print(reference)

candidate = ' '.join((result_df['gpt2_cleaned'][5498])).split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate, weights=(bleu_weight[0], bleu_weight[1], bleu_weight[2], bleu_weight[3]))))
print(candidate)

[['hybe', 'hybe', 'labels']]
BLEU score -> 0
['no_tag']


## **BLEU Score Computation**

In [ ]:
def compute_bleu(row, col):
  global bleu_weight
  reference = [' '.join(row['top_tags'][:10]).split()]
  if row[col] == None:
    score = 0
  else:
    try:
      candidate = ' '.join((row[col])).split()
    except:
      return 0
    score = sentence_bleu(reference, candidate, weights=(bleu_weight[0], bleu_weight[1], bleu_weight[2], bleu_weight[3]))
  return score


In [ ]:
import warnings
bleu_score = []
with warnings.catch_warnings():
  warnings.simplefilter('ignore')
  target = ['dnn_512','dnn_768','dnn_1024','gpt2_cleaned','vicuna13b_cleaned']
  for col in target:
    score = result_df.apply(compute_bleu,axis=1, args=(col,)).mean()
    bleu_score.append(score)
    print("BLEU Score of " + col + " is: " + str(score))

BLEU Score of dnn_512 is: 0.08514112433723528
BLEU Score of dnn_768 is: 0.1075136772973815
BLEU Score of dnn_1024 is: 0.08352298041592456
BLEU Score of gpt2_cleaned is: 0.11847727182048118
BLEU Score of vicuna13b_cleaned is: 0.023311012454593983


In [ ]:
px.bar(pd.DataFrame(bleu_score, index=target)).update_layout(xaxis_title="Model", yaxis_title="BLEU Score", width=800)

# **Precision Anaylsis**

In [ ]:
result_df_prec = result_df.copy()
result_df_prec.head()

,combined,top_tags,dnn_512,dnn_768,dnn_1024,gpt2,vicuna13b,gpt2_cleaned,vicuna13b_cleaned
0,love.Provided to YouTube by Universal Music Gr...,[kid cudi],"[pop, music, records, savage, columbia, music ...","[kid cudi, boy better know, savage, spillage v...","[columbia, young thug, records, potter payper,...",['kid cudi'],"['love', 'Kid Cudi', 'The Boy Who Flew To The ...",[kid cudi],"[love, kid cudi, the boy who flew to the moon,..."
1,Pope & Root Masterclass! | Highlights | Englan...,"[england, england vs. new zealand, new zealand...","[england cricket, jonny bairstow, england cric...","[new zealand, england, cricket highlights, joe...","[england cricket board, cricket, cricket video...","['england', 'england vs. new zealand', 'new ze...","['cricket', 'test match', 'england cricket', '...","[england, england vs. new zealand, new zealand...","[cricket, test match, england cricket, new zea..."
2,telling you the truth.this is the truth. ☂️ Sh...,"[fortnite, fortnite battle royale, competitive...","[sypherpk, fortnite highlights, fortnite tips,...","[fortnite tips, sypherpk, fortnite highlights,...","[sypherpk, sypher, fortnite tips, how to play,...","['sypherpk', 'sypherpk', 'fortnite', 'sypherpk...","['sypherpk', 'fortnite', 'battle royale', 'gam...","[sypherpk, sypherpk, fortnite, sypherpk fortni...","[sypherpk, fortnite, battle royale, gaming, vl..."
3,Basketball Mood Lifters! Part 7! 😆 #shortsBask...,"[shorts, youtube shorts, tiktok, viral, instag...","[ballislife, 2hype, jdub, basketball, kenny ch...","[nba, dunk, ballislife, basketball, tiktoker, ...","[kenny chao, ballislife, 2hype, 2k, jdub, gio ...","['dub', 'dub team', 'nba 2k', 'basketball', 'n...","['basketball', 'dunk', 'sports', 'viral', 'ins...","[dub, dub team, nba 2k, basketball, nba 2k22, ...","[basketball, dunk, sports, viral, instagram, r..."
4,BRYNLEY IS PREGNANT!BRYNLEY IS PREGNANT!Subscr...,"[first pregnancy, fertility, baby announcement...","[shorts, the ace family, trending, family, com...","[christian youtuber, natural labor, pranks, ru...","[london, vlogs, vlog, christian youtuber, mr b...","['family', 'shorts', 'shorts', 'shorts', 'lind...","['The Arnold Sisters', 'pregnancy announcement...","[family, shorts, shorts, shorts, lindian, the ...","[the arnold sisters, pregnancy announcement, d..."


In [ ]:
#calculate the weight
def cal_prec(label,pred):
  weight = 1
  if len(label) == 0:
    return 0
  else:
    if len(label) <= 10:
      weight = 10/len(label)
    if len(label) >10:
      label = label[:10]

    count = 0
    try:
      for word in pred:
        if word in label:
          count += 1
      prec = count/len(pred) * weight
      return prec
    except:
      return 0

result_df_prec['dnn_512_prec'] = result_df_prec.apply(lambda x: cal_prec(x['top_tags'], x['dnn_512']), axis=1)
result_df_prec['dnn_768_prec'] = result_df_prec.apply(lambda x: cal_prec(x['top_tags'], x['dnn_768']), axis=1)
result_df_prec['dnn_1024_prec'] = result_df_prec.apply(lambda x: cal_prec(x['top_tags'], x['dnn_1024']), axis=1)
result_df_prec['gpt2_prec'] = result_df_prec.apply(lambda x: cal_prec(x['top_tags'], x['gpt2_cleaned']), axis=1)
result_df_prec['vicuna13b_prec'] = result_df_prec.apply(lambda x: cal_prec(x['top_tags'], x['vicuna13b_cleaned']), axis=1)

In [ ]:
result_df_prec[['top_tags','gpt2_cleaned','vicuna13b_cleaned']].head()

,top_tags,gpt2_cleaned,vicuna13b_cleaned
0,[kid cudi],[kid cudi],"[love, kid cudi, the boy who flew to the moon,..."
1,"[england, england vs. new zealand, new zealand...","[england, england vs. new zealand, new zealand...","[cricket, test match, england cricket, new zea..."
2,"[fortnite, fortnite battle royale, competitive...","[sypherpk, sypherpk, fortnite, sypherpk fortni...","[sypherpk, fortnite, battle royale, gaming, vl..."
3,"[shorts, youtube shorts, tiktok, viral, instag...","[dub, dub team, nba 2k, basketball, nba 2k22, ...","[basketball, dunk, sports, viral, instagram, r..."
4,"[first pregnancy, fertility, baby announcement...","[family, shorts, shorts, shorts, lindian, the ...","[the arnold sisters, pregnancy announcement, d..."


In [ ]:
result_df_prec[['dnn_512_prec','dnn_768_prec','dnn_1024_prec','gpt2_prec','vicuna13b_prec']].mean()

dnn_512_prec      0.350956
dnn_768_prec      0.452545
dnn_1024_prec     0.362094
gpt2_prec         0.401244
vicuna13b_prec    0.190122
dtype: float64

In [ ]:
px.bar(result_df_prec[['dnn_512_prec','dnn_768_prec','dnn_1024_prec','gpt2_prec','vicuna13b_prec']].mean()).update_layout(xaxis_title="Model", yaxis_title="Precision", width=800)

# **Latent Semantic Analysis**

E5 Large v2 is the SOTA embedding model in Hugging Face MTEB Leaderboard

https://huggingface.co/spaces/mteb/leaderboard



In [ ]:
### Reference https://huggingface.co/intfloat/e5-large-v2

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large-v2')
model = AutoModel.from_pretrained('intfloat/e5-large-v2')

In [ ]:
%%time

### Reference https://huggingface.co/intfloat/e5-large-v2

def embed(text):
  batch_dict = tokenizer(text, max_length=1024, padding=True, truncation=True, return_tensors='pt')
  outputs = model(**batch_dict)
  embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
  embeddings = F.normalize(embeddings, p=2, dim=1)
  return embeddings.detach().numpy()

embed('what are you doing').shape

CPU times: user 526 ms, sys: 5.49 ms, total: 531 ms
Wall time: 90.6 ms


(1, 1024)

In [ ]:
embed('what are you doing')

array([[ 0.02609257, -0.05705491,  0.01673909, ..., -0.01882523,
         0.02301931,  0.02364756]], dtype=float32)

In [ ]:
result_df[['top_tags','dnn_512','dnn_768','dnn_1024','gpt2_cleaned','vicuna13b_cleaned']]

,top_tags,dnn_512,dnn_768,dnn_1024,gpt2_cleaned,vicuna13b_cleaned
0,[kid cudi],"[pop, music, records, savage, columbia, music ...","[kid cudi, boy better know, savage, spillage v...","[columbia, young thug, records, potter payper,...",[kid cudi],"[love, kid cudi, the boy who flew to the moon,..."
1,"[england, england vs. new zealand, new zealand...","[england cricket, jonny bairstow, england cric...","[new zealand, england, cricket highlights, joe...","[england cricket board, cricket, cricket video...","[england, england vs. new zealand, new zealand...","[cricket, test match, england cricket, new zea..."
2,"[fortnite, fortnite battle royale, competitive...","[sypherpk, fortnite highlights, fortnite tips,...","[fortnite tips, sypherpk, fortnite highlights,...","[sypherpk, sypher, fortnite tips, how to play,...","[sypherpk, sypherpk, fortnite, sypherpk fortni...","[sypherpk, fortnite, battle royale, gaming, vl..."
3,"[shorts, youtube shorts, tiktok, viral, instag...","[ballislife, 2hype, jdub, basketball, kenny ch...","[nba, dunk, ballislife, basketball, tiktoker, ...","[kenny chao, ballislife, 2hype, 2k, jdub, gio ...","[dub, dub team, nba 2k, basketball, nba 2k22, ...","[basketball, dunk, sports, viral, instagram, r..."
4,"[first pregnancy, fertility, baby announcement...","[shorts, the ace family, trending, family, com...","[christian youtuber, natural labor, pranks, ru...","[london, vlogs, vlog, christian youtuber, mr b...","[family, shorts, shorts, shorts, lindian, the ...","[the arnold sisters, pregnancy announcement, d..."
...,...,...,...,...,...,...
5494,"[emmanuel macron, guerre ukraine, information,...","[sketch, humour, fun, clip, video, music, scen...","[information, russie, ukraine russie, tf1, fra...","[cinema, video drole, tournoi, inox, horreur, ...","[information, lci, russie, cartoon, animation,...","[politics, interview, french president, emmanu..."
5495,"[soolking, soolking officiel, soolking 2020, s...","[sofiane, dadju, zemer, melegim, soolking paro...","[soolking officiel, zemer, soolking parole, so...","[dadju, soolking album, soolking parole, soolk...","[soolking, jennifer, ]","[soolking, lynda, heuss, l'algérino, franglish..."
5496,"[fox business, fox business live, fox business...","[news, world news, politics, donald trump, joe...","[politics, donald trump, crypto, joe biden, st...","[stock market, cable, breaking news, cable new...","[fox business, finance stock, stock market, ne...","[rnc, republican national convention, trump, a..."
5497,"[spittin chiclets, chiclets, hockey, nhl, ryan...","[sports, podcast, hockey, barstool sports, nhl...","[comedy, barstool golf, biz, sports, paul biss...","[comedy, sports, barstool sports, podcast, bar...","[spittin chiclets, barstool sports, mr beast, ...","[barstool sports, grilling competition, ryan w..."


In [ ]:
result_df[['gpt2_cleaned','gpt2']].head(30)

,gpt2_cleaned,gpt2
0,[kid cudi],['kid cudi']
1,"[england, england vs. new zealand, new zealand...","['england', 'england vs. new zealand', 'new ze..."
2,"[sypherpk, sypherpk, fortnite, sypherpk fortni...","['sypherpk', 'sypherpk', 'fortnite', 'sypherpk..."
3,"[dub, dub team, nba 2k, basketball, nba 2k22, ...","['dub', 'dub team', 'nba 2k', 'basketball', 'n..."
4,"[family, shorts, shorts, shorts, lindian, the ...","['family', 'shorts', 'shorts', 'shorts', 'lind..."
5,"[sza, hit different, new sza, sza is back, ctr...","['sza', 'hit different', 'new sza', 'sza is ba..."
6,[no_tag],['no_tag']
7,None,"['stormzy',"
8,"[hermitcraft, minecraft, hermitcraft, minecraf...","['hermitcraft', 'minecraft', 'hermitcraft', 'm..."
9,"[spacex, starship, boca chica, testing, bocach...","['spacex', 'starship', 'boca chica', 'testing'..."


In [ ]:
def lst_emb(lst):
  try:
    return embed(' '.join(lst[:10]))
  except:
    return embed('no_tag')

In [ ]:
%%time

target = ['dnn_512','dnn_768','dnn_1024','gpt2_cleaned','vicuna13b_cleaned']
embedding_df = pd.DataFrame()

for col in target:
  print('Started transforming ' + col + ' to embedding')
  embedding_df[col] = result_df[col].apply(lst_emb)
  print('Completed transforming ' + col + ' to embedding')

embedding_df

Started transforming dnn_512 to embedding
Completed transforming dnn_512 to embedding
Started transforming dnn_768 to embedding
Completed transforming dnn_768 to embedding
Started transforming dnn_1024 to embedding
Completed transforming dnn_1024 to embedding
Started transforming gpt2_cleaned to embedding
Completed transforming gpt2_cleaned to embedding
Started transforming vicuna13b_cleaned to embedding
Completed transforming vicuna13b_cleaned to embedding
CPU times: user 7h 4min 43s, sys: 21.8 s, total: 7h 5min 5s
Wall time: 1h 10min 47s


,dnn_512,dnn_768,dnn_1024,gpt2_cleaned,vicuna13b_cleaned
0,"[[-0.013002924, -0.056688208, 0.035415366, 0.0...","[[0.0019918028, -0.037250876, 0.0315204, 0.016...","[[0.014816376, -0.06794276, 0.028569674, 0.004...","[[0.033927225, -0.071698725, 0.009496503, -0.0...","[[0.013051257, -0.037567656, 0.027977655, 0.00..."
1,"[[0.022323227, -0.049390893, 0.028588539, -0.0...","[[0.030211905, -0.048821714, 0.026561968, -0.0...","[[0.03382722, -0.04311082, 0.030644767, -0.016...","[[0.029041858, -0.03688354, -0.0042592497, -0....","[[0.010367792, -0.047555853, 0.0070514735, 0.0..."
2,"[[0.04820295, -0.053893548, 0.018265445, -0.02...","[[0.04074863, -0.049980927, 0.02176576, -0.016...","[[0.032759458, -0.04728796, 0.013325319, -0.01...","[[0.026521103, -0.038602404, 0.0031585633, -0....","[[0.017937124, -0.060931124, 0.007079502, -0.0..."
3,"[[0.03703556, -0.031848114, 0.050961144, -0.00...","[[0.032730203, -0.034555186, 0.031174036, 0.00...","[[0.03538184, -0.047443904, 0.040639453, -0.00...","[[0.04463327, -0.05555959, 0.03476546, -0.0131...","[[0.018492246, -0.03988894, 0.043574613, -0.03..."
4,"[[0.024155775, -0.065468885, 0.050815277, 0.01...","[[-0.008049404, -0.051794875, 0.009524048, -0....","[[-0.0017703084, -0.045910656, 0.025979018, -0...","[[0.024477305, -0.06389803, 0.04117537, 0.0049...","[[-0.007799906, -0.032756366, 0.051691864, -0...."
...,...,...,...,...,...
5494,"[[0.0134856915, -0.037548937, 0.043414038, -0....","[[0.021216044, -0.046797264, 0.013620391, -0.0...","[[0.014853777, -0.05774846, 0.036438074, 0.009...","[[-0.021401394, -0.043353673, 0.024277825, -0....","[[0.014177988, -0.014828813, 0.019745983, -0.0..."
5495,"[[-0.0057802997, -0.057371017, 0.0351267, 0.01...","[[-0.0056466176, -0.06331629, 0.049040426, -0....","[[-0.009605567, -0.055220697, 0.032472648, 0.0...","[[-0.002885676, -0.057223186, 0.038492545, -0....","[[0.015479875, -0.03540986, 0.06866938, 0.0077..."
5496,"[[0.014792125, -0.051439025, 0.02520019, -0.03...","[[0.013237626, -0.052761428, 0.03241597, -0.01...","[[0.020759318, -0.028985009, 0.027107421, -0.0...","[[0.010547962, -0.053677764, 0.027752263, -0.0...","[[0.005055341, -0.04482195, 0.016792929, -0.03..."
5497,"[[0.01786784, -0.013111469, 0.034987587, -0.01...","[[0.022465209, -0.029778132, 0.04366789, -0.00...","[[0.01083458, -0.04286829, 0.04779841, -0.0113...","[[0.01647301, -0.04612717, 0.036943857, -0.015...","[[-0.00073318486, -0.052204832, 0.032212697, 0..."


In [ ]:
embedding_df['top_tags'] = result_df['top_tags'].apply(lst_emb)
embedding_df

,dnn_512,dnn_768,dnn_1024,gpt2_cleaned,vicuna13b_cleaned,top_tags
0,"[[-0.013002924, -0.056688208, 0.035415366, 0.0...","[[0.0019918028, -0.037250876, 0.0315204, 0.016...","[[0.014816376, -0.06794276, 0.028569674, 0.004...","[[0.033927225, -0.071698725, 0.009496503, -0.0...","[[0.013051257, -0.037567656, 0.027977655, 0.00...","[[0.033927225, -0.071698725, 0.009496503, -0.0..."
1,"[[0.022323227, -0.049390893, 0.028588539, -0.0...","[[0.030211905, -0.048821714, 0.026561968, -0.0...","[[0.03382722, -0.04311082, 0.030644767, -0.016...","[[0.029041858, -0.03688354, -0.0042592497, -0....","[[0.010367792, -0.047555853, 0.0070514735, 0.0...","[[0.036725987, -0.042395823, 0.001507096, -0.0..."
2,"[[0.04820295, -0.053893548, 0.018265445, -0.02...","[[0.04074863, -0.049980927, 0.02176576, -0.016...","[[0.032759458, -0.04728796, 0.013325319, -0.01...","[[0.026521103, -0.038602404, 0.0031585633, -0....","[[0.017937124, -0.060931124, 0.007079502, -0.0...","[[0.016059637, -0.06639106, -0.0072544804, -0...."
3,"[[0.03703556, -0.031848114, 0.050961144, -0.00...","[[0.032730203, -0.034555186, 0.031174036, 0.00...","[[0.03538184, -0.047443904, 0.040639453, -0.00...","[[0.04463327, -0.05555959, 0.03476546, -0.0131...","[[0.018492246, -0.03988894, 0.043574613, -0.03...","[[0.028479325, -0.042851586, 0.04404274, -0.02..."
4,"[[0.024155775, -0.065468885, 0.050815277, 0.01...","[[-0.008049404, -0.051794875, 0.009524048, -0....","[[-0.0017703084, -0.045910656, 0.025979018, -0...","[[0.024477305, -0.06389803, 0.04117537, 0.0049...","[[-0.007799906, -0.032756366, 0.051691864, -0....","[[0.010479151, -0.052209493, 0.055771094, 0.00..."
...,...,...,...,...,...,...
5494,"[[0.0134856915, -0.037548937, 0.043414038, -0....","[[0.021216044, -0.046797264, 0.013620391, -0.0...","[[0.014853777, -0.05774846, 0.036438074, 0.009...","[[-0.021401394, -0.043353673, 0.024277825, -0....","[[0.014177988, -0.014828813, 0.019745983, -0.0...","[[0.015878577, -0.018784402, 0.0008154297, -0...."
5495,"[[-0.0057802997, -0.057371017, 0.0351267, 0.01...","[[-0.0056466176, -0.06331629, 0.049040426, -0....","[[-0.009605567, -0.055220697, 0.032472648, 0.0...","[[-0.002885676, -0.057223186, 0.038492545, -0....","[[0.015479875, -0.03540986, 0.06866938, 0.0077...","[[-0.0014199842, -0.05877662, 0.020655345, 0.0..."
5496,"[[0.014792125, -0.051439025, 0.02520019, -0.03...","[[0.013237626, -0.052761428, 0.03241597, -0.01...","[[0.020759318, -0.028985009, 0.027107421, -0.0...","[[0.010547962, -0.053677764, 0.027752263, -0.0...","[[0.005055341, -0.04482195, 0.016792929, -0.03...","[[0.0314137, -0.055148456, 0.023827173, -0.031..."
5497,"[[0.01786784, -0.013111469, 0.034987587, -0.01...","[[0.022465209, -0.029778132, 0.04366789, -0.00...","[[0.01083458, -0.04286829, 0.04779841, -0.0113...","[[0.01647301, -0.04612717, 0.036943857, -0.015...","[[-0.00073318486, -0.052204832, 0.032212697, 0...","[[0.023076234, -0.04139681, 0.05001564, -0.003..."


In [ ]:
a = embed(' '.join(result_df['top_tags'][1]))
b = embed(' '.join(result_df['gpt2_cleaned'][1]))

def vector_similarity(v1,v2,method='cosine'):
  if method == 'dot':
    score = np.inner(v1, v2)
  if method == 'cosine':
    score = np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
  if method == 'distance':
    score = np.linalg.norm(v1 - v2)
  return score

vector_similarity(a[0],b[0])

0.9626424

## **Compute Cosine SImilarity**

In [ ]:
similarity_df = pd.DataFrame()

target = ['dnn_512','dnn_768','dnn_1024','gpt2_cleaned','vicuna13b_cleaned']

for col in target:
  similarity_df[col] = embedding_df.apply(lambda x: vector_similarity(x['top_tags'][0], x[col][0]), axis=1)

similarity_df

,dnn_512,dnn_768,dnn_1024,gpt2_cleaned,vicuna13b_cleaned
0,0.817240,0.826299,0.824639,1.000000,0.833449
1,0.892483,0.915142,0.834383,0.975278,0.876019
2,0.911658,0.880506,0.890800,0.840785,0.837143
3,0.818737,0.878894,0.813178,0.811452,0.929273
4,0.767785,0.774637,0.740316,0.758536,0.887910
...,...,...,...,...,...
5494,0.777505,0.846139,0.767993,0.797591,0.890587
5495,0.889468,0.925079,0.886240,0.850010,0.837275
5496,0.819030,0.831293,0.822444,0.848457,0.912200
5497,0.946203,0.951175,0.930769,0.894343,0.923164


In [ ]:
similarity_df.describe()

,dnn_512,dnn_768,dnn_1024,gpt2_cleaned,vicuna13b_cleaned
count,5499.000000,5499.000000,5499.000000,5499.000000,5499.000000
mean,0.854737,0.876087,0.857332,0.837083,0.849061
std,0.069780,0.065048,0.068910,0.087185,0.047270
min,0.689129,0.692949,0.672804,0.668181,0.679634
25%,0.795869,0.827973,0.799353,0.753762,0.820657
50%,0.849032,0.883819,0.854861,0.834074,0.852637
75%,0.911325,0.928283,0.911728,0.897023,0.882201
max,0.998032,0.995073,0.997508,1.000000,0.980789


In [ ]:
px.bar(pd.DataFrame(similarity_df.describe().T['mean'], index=target)).update_layout(xaxis_title="Model", yaxis_title="Cosine Similarity", width=800)

# **Compare Models in Different Metrics**

In [ ]:
vis_df = pd.DataFrame()
target = ['dnn_512', 'dnn_768', 'dnn_1024', 'gpt2_fine_tuned', 'vicuna13b']
vis_df['BLEU'] = pd.Series(bleu_score, index=target)
vis_df['Precision'] = result_df_prec[['dnn_512_prec','dnn_768_prec','dnn_1024_prec','gpt2_prec','vicuna13b_prec']].mean().set_axis(target, axis=0)
vis_df['Cosine Similarity'] = similarity_df.describe().T['mean'].set_axis(target, axis=0)

vis_df = vis_df.stack().reset_index().rename(columns={"level_0": "Model", "level_1": "Metrics", 0: "Score"})

In [ ]:
px.histogram(vis_df, x="Metrics", y="Score",
             color='Model', barmode='group',
             height=500, width=600).update_layout(yaxis_title="Score")